## Variable Selection: Best Subset 

Let's try out the best subset selection algorithm using the baseball salary data we have worked with previously. Our goal is still to predict a player’s `Salary` using some combination of the performance statistics included in the file.


In [1]:
%matplotlib inline
import itertools
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://www4.stat.ncsu.edu/~boos/var.select/baseball.txt' #download this file using wget and load locally
df = pd.read_csv(url)
df.rename(columns={'x1': 'Batting average', 'x2': 'On-base percentage', 'x3': 'Number of runs', 'x4': 'Number of hits', 'x5': 'Number of doubles', 'x6': 'Number of triples', 'x7': 'Number of home runs', 'x8': 'Number of runs batted in', 'x9': 'Number of walks', 'x10': 'Number of strike-outs', 'x11': 'Number of stolen bases', 'x12': 'Number of errors', 'x13': 'FAeligibility', 'x14': 'Indicator of "free agent in 1991/2"', 'x15': 'Indicator of "arbitration eligibility"', 'x16': 'Indicator of "arbitration in 1991/2"', 'y': 'Salary'}, inplace=True)
print(df)

     Salary  Batting average  On-base percentage  Number of runs  \
0      3300            0.272               0.302              69   
1      2600            0.269               0.335              58   
2      2500            0.249               0.337              54   
3      2475            0.260               0.292              59   
4      2313            0.273               0.346              87   
..      ...              ...                 ...             ...   
332     170            0.111               0.138               3   
333     160            0.264               0.318              24   
334     142            0.187               0.281              38   
335     140            0.264               0.270              24   
336     109            0.258               0.395               6   

     Number of hits  Number of doubles  Number of triples  \
0               153                 21                  4   
1               111                 17                  2   


In [2]:
print("Number of null values:", df.isnull().sum())

Number of null values: Salary                                    0
Batting average                           0
On-base percentage                        0
Number of runs                            0
Number of hits                            0
Number of doubles                         0
Number of triples                         0
Number of home runs                       0
Number of runs batted in                  0
Number of walks                           0
Number of strike-outs                     0
Number of stolen bases                    0
Number of errors                          0
FAeligibility                             0
Indicator of "free agent in 1991/2"       0
Indicator of "arbitration eligibility"    0
Indicator of "arbitration in 1991/2"      0
dtype: int64


In [3]:
# Print the dimensions of the data (337 rows x 17 columns)
print("Dimensions of original data:", df.shape)

Dimensions of original data: (337, 17)


We have measurements along every dimension for each observational unit. That is good news - less cleaning and more analyzing! We can now move on to immediately apply the best subset selection algorithm. This algorithm will identify the 'best' model  containing a specified number of predictors. How 'best' is quantified is up to the user. It could be decided according to  $\bar{R^2}$, AIC, BIC, or RSS. To start we will use RSS as our metric. We can also define a helper function that will generate the best set of variables for each model size:

In [4]:

y = df.Salary

# Drop the column with the independent variable (Salary)
X = df.drop(['Salary'], axis=1).astype('float64')



In [5]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [6]:
def process(feature_set):
    # Fit model on feature_set and calculate aic
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    return {"model":regr, "AIC":regr.aic}

In [7]:
model = sm.OLS(y,X[['Batting average','Number of runs']])
regr = model.fit()
regr.params

Batting average   -150.802471
Number of runs      27.566824
dtype: float64

In [8]:
def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the smallest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [9]:
def getBested(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(process(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the smallest RSS
    best_model = models.loc[models['AIC'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [10]:
models_best = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,8):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed 16 models on 1 predictors in 0.05915427207946777 seconds.
Processed 120 models on 2 predictors in 0.2243938446044922 seconds.
Processed 560 models on 3 predictors in 1.0157053470611572 seconds.
Processed 1820 models on 4 predictors in 3.4091951847076416 seconds.
Processed 4368 models on 5 predictors in 8.108511686325073 seconds.
Processed 8008 models on 6 predictors in 14.84723949432373 seconds.
Processed 11440 models on 7 predictors in 21.991337060928345 seconds.
Total elapsed time: 49.86320972442627 seconds.


In [11]:
models_bested = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,8):
    models_bested.loc[i] = getBested(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed 16 models on 1 predictors in 0.018270492553710938 seconds.
Processed 120 models on 2 predictors in 0.11421751976013184 seconds.
Processed 560 models on 3 predictors in 0.5156097412109375 seconds.
Processed 1820 models on 4 predictors in 1.6911451816558838 seconds.
Processed 4368 models on 5 predictors in 4.715756177902222 seconds.
Processed 8008 models on 6 predictors in 8.461341381072998 seconds.
Processed 11440 models on 7 predictors in 12.785317182540894 seconds.
Total elapsed time: 28.53597068786621 seconds.


In [ ]:
results = []

for k in range(1,17):
    for combo in itertools.combinations(X.columns, k):
        results.append(process(combo))

In [ ]:
print(getBested(2)["model"].summary())


In [ ]:
print(models_best.loc[4, "model"].summary())

Excellent! In addition to the verbose output we get when we print the summary to the screen, fitting the OLM also produced many other useful statistics such as $\bar{R^2}$ , AIC, and BIC. We can examine these to try to select the best overall model. Let's start by looking at  R2  across all our models:

In [ ]:
models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 18, 'lines.markersize': 10})

# Set up a 2x2 grid so we can look at 4 plots at once
plt.subplot(2, 2, 1)

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector
plt.plot(models_best["RSS"])
plt.xlabel('# Predictors')
plt.ylabel('RSS')

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector

rsquared_adj = models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

plt.subplot(2, 2, 2)
plt.plot(rsquared_adj)
plt.plot(rsquared_adj.argmax(), rsquared_adj.max(), "or")
plt.xlabel('# Predictors')
plt.ylabel('adjusted rsquared')

# We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
aic = models_best.apply(lambda row: row[1].aic, axis=1)

plt.subplot(2, 2, 3)
plt.plot(aic)
plt.plot(aic.argmin(), aic.min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('AIC')

bic = models_best.apply(lambda row: row[1].bic, axis=1)

plt.subplot(2, 2, 4)
plt.plot(bic)
plt.plot(bic.argmin(), bic.min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('BIC')

## Forward Stepwise Selection

In [ ]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
models_fwd = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
predictors = []

for i in range(1,len(X.columns)+1):    
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

In [ ]:
print(models_fwd.loc[1, "model"].summary())
print(models_fwd.loc[2, "model"].summary())

In [ ]:
print(models_best.loc[6, "model"].summary())
print(models_fwd.loc[6, "model"].summary())

## Backwards Stepwise Selection

In [ ]:
def backward(predictors):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
models_bwd = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(X.columns)))

tic = time.time()
predictors = X.columns

while(len(predictors) > 1):  
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

For this data, the best one-variable through six-variable models are each identical for best subset and forward selection. However, the best seven-variable models identified by forward stepwise selection, backward stepwise selection, and best subset selection are different:

In [ ]:
print("------------")
print("Best Subset:")
print("------------")
print(models_best.loc[7, "model"].params)

In [ ]:
print("-----------------")
print("Foward Selection:")
print("-----------------")
print(models_fwd.loc[7, "model"].params)

In [ ]:
print("-------------------")
print("Backward Selection:")
print("-------------------")
print(models_bwd.loc[7, "model"].params)